# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
         T-statistics is appropirate for this problem because the population variance is unkown. Also, permutation test is alse appropriate because this test is desinged for testing the null hypothesis of no difference between two groups. 
         The CLT apply because the sample size is 4870, which is large enough.
   2. What are the null and alternate hypotheses? Null hypothesis is that there is no significant difference of callback rate between black and white. The alternate hypethesis is that there is a significant difference of callback rate between black and white.
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. The margin of error is 0.015. The confidence interval is within the range [0.01677, 0.0473]. By using bootstrapping approach, the p-value is 0.0002. By using frequentist approach, the p-value is even less, much closer to zero.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
   Our analysis while statistically shows that whites and blacks definitely don't have the same callback rate because of the p-value being less than 0.05, it does not conclude that race/name is the most important factor in callback success, because we have not looked at other features like candiate history, criminal records, etc. Hence, we can conclude that there is a significant difference between race/name and callback success.
You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

#Permutation tests work well for the null hypothesis of no difference between treatment groups. To do the two-sample bootstrap test, I perform a bootstrap test of the hypothesis that males and females have identical distribution of temperatures using the difference of means test statistics.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy.stats as st
import scipy.special as sp

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [14]:
len(data)

4870

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

  What test is appropriate for this problem? Does CLT apply?
         Both permutation test and t-statistics is appropirate for this problem.
         I do not think the CLT apply because the sample can be divided into two groups.

  What are the null and alternate hypothesis?
     null hypothesis: call back rates are equal for whites and black
     alternate hypothesis: call back rates are not equal for whites and black

In [5]:
# Your solution to Q3 here

In [6]:
whites = data[data.race=='w']['call']
blacks = data[data.race=='b']['call']


In [7]:
# frequentist statistical approach : t-statistics
mu_w = whites.mean()
var_w = whites.var()
mu_b = blacks.mean()
var_b = blacks.var()
t = (mu_w - mu_b)/np.sqrt(var_w/len(whites)+var_b/len(blacks))
pval_t=st.norm.sf(abs(t)) * 2 
pval_t

3.876188070108842e-05

In [17]:
# Boostrapping Approach - permutation test
def diff_of_means(data_1, data_2):
    a = np.mean(data_1)- np.mean(data_2)
    return a


empirical_diff_means= diff_of_means(whites, blacks)
race_concat = np.concatenate([whites, blacks])
bs_replicates = np.empty(10000)
for i in range(10000):
    bs_sample = np.random.choice(race_concat, size = len(race_concat))
    bs_replicates[i] = diff_of_means(bs_sample[: len(whites)], bs_sample[len(blacks):])
p= np.sum(bs_replicates >= empirical_diff_means)/10000
p

0.0002

In [9]:
se = np.sqrt(var_w/len(whites)+var_b/len(blacks))
se

0.007784906826883674

In [10]:
diff = mu_w - mu_b
low_conf_int = diff - 1.96*se 
high_conf_int = diff + 1.96*se 
print('Confidence Intervals: ', low_conf_int, high_conf_int)

Confidence Intervals:  0.016774437474368575 0.04729127223575258


In [11]:
moe = se * 1.96
moe

0.015258417380692

In [13]:
w = data[data.race=='w']
b = data[data.race=='b']

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Our analysis while statistically shows that whites and blacks definitely don't have the same callback rate because of the p-value being less than 0.05, it does not conclude that race/name is the most important factor in callback success, because we have not looked at other features like candiate history, criminal records, etc. Hence, we can conclude that there is a significant difference between race/name and callback success.